<a href="https://colab.research.google.com/github/Aeagon07/Pytourch/blob/main/RNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd

# Step 1: Upload file
uploaded = files.upload()

# Step 2: Check the uploaded filenames
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

# Step 3: Load into pandas dataframe (if it's a CSV)
df = pd.read_csv(next(iter(uploaded)))
df.head()

Saving 100_Unique_QA_Dataset.csv to 100_Unique_QA_Dataset.csv
User uploaded file "100_Unique_QA_Dataset.csv" with length 4284 bytes


,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
#  Tokenize
def tokenize(text):
  text = text.lower()
  text.replace(',', ' ')
  text.replace('.', ' ')
  text.replace('?', ' ')
  text.replace("'", " ")
  return text.split()
# ['what', 'is', 'the', 'capital', 'of', 'france'] -> it look like this..

In [4]:
# Vocab
vocab = {'<UNK>': 0}

In [5]:
def build_vocab(data):
  print(data['question'], data['answer'])
  tok_ques = tokenize(data['question'])
  tok_ans = tokenize(data['answer'])

  # Now merge the both list
  merge_list = tok_ques + tok_ans

  # Now run the loop on the merged list
  for token in merge_list:
    if token not in vocab:
      vocab[token] = len(vocab) # what = 1, is = 2, ..
      # From the current length of vacab is the number of the each next unique word

In [6]:
df.apply(build_vocab, axis=1) # axis=1 for complete data in each will process

What is the capital of France? Paris
What is the capital of Germany? Berlin
Who wrote 'To Kill a Mockingbird'? Harper-Lee
What is the largest planet in our solar system? Jupiter
What is the boiling point of water in Celsius? 100
Who painted the Mona Lisa? Leonardo-da-Vinci
What is the square root of 64? 8
What is the chemical symbol for gold? Au
Which year did World War II end? 1945
What is the longest river in the world? Nile
What is the capital of Japan? Tokyo
Who developed the theory of relativity? Albert-Einstein
What is the freezing point of water in Fahrenheit? 32
Which planet is known as the Red Planet? Mars
Who is the author of '1984'? George-Orwell
What is the currency of the United Kingdom? Pound
What is the capital of India? Delhi
Who discovered gravity? Newton
How many continents are there on Earth? 7
Which gas do plants use for photosynthesis? CO2
What is the smallest prime number? 2
Who invented the telephone? Alexander-Graham-Bell
What is the capital of Australia? Canber

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [7]:
len(vocab)

337

In [8]:
# convert word to index
def word_to_index(data, vocab):

  index_text = []

  for token in tokenize(data):

    if token in vocab:
      # when in the vocab every word has it's index, we take the first word what search it in vocab take the index we given and append it in index_text

      index_text.append(vocab[token])

      # After, this our englist statement which looks like ['1', '2', '3', '4'] in this formate
    else:
      index_text.append(vocab['<UNK>'])

  return index_text

In [9]:
word_to_index("what is CampusX", vocab) # what = 1, is = 2 and campusX is not present in vocab so CampusX = 0

[1, 2, 0]

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_Q = word_to_index(self.df.iloc[index]['question'], vocab)
    numerical_A = word_to_index(self.df.iloc[index]['answer'], vocab)

    return torch.tensor(numerical_Q), torch.tensor(numerical_A)

In [11]:
dataset = QADataset(df, vocab)

dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [12]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [13]:
for question, ans in dataloader:
  print(question, ans)

tensor([[ 42,   2,   3, 217, 142, 175, 218, 176]]) tensor([[219]])
tensor([[  1,   2,   3, 106,   5, 107,  19, 108]]) tensor([[109]])
tensor([[80, 81, 82, 83, 84, 85, 86]]) tensor([[87]])
tensor([[ 10,  77, 114]]) tensor([[115]])
tensor([[  1,   2,   3,   4,   5, 247, 248]]) tensor([[249]])
tensor([[ 42, 142,   2, 143,  39, 144]]) tensor([[145]])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([[16]])
tensor([[  1,   2,   3,  17, 118,  85,  86]]) tensor([[119]])
tensor([[ 42, 224, 121, 225, 226,  19,  14, 227, 228]]) tensor([[229]])
tensor([[  1,   2,   3, 187, 188, 189, 190]]) tensor([[191]])
tensor([[ 42, 103,   2,   3, 104]]) tensor([[105]])
tensor([[ 10, 320,   3, 321, 322]]) tensor([[323]])
tensor([[ 1,  2,  3,  4,  5, 75]]) tensor([[76]])
tensor([[ 10,   2,  63,  64,   3, 295,   5, 296]]) tensor([[297]])
tensor([[ 10, 146,   3, 147, 282,  95, 283,   5,   3, 284]]) tensor([[285]])
tensor([[ 10,  98,   3, 107, 250]]) tensor([[251]])
tensor([[  1,   2,   3,   4,   5, 291]]) tensor([[292]]

<b>Summary</b>

till now you did =>
1) Craete tokenization function by doing the splits and remove the extra symbol like ?, '
2) Build the vocab by first convert the question and answer to tokenized form then merge them and loop through to check if they present in vocab or not if not then just add in vocab give the index number as the vocab length at that time.
3) After apply these features on the data now you create the word to indix function and convert the question and answers into the numerical form using vocab cause they given the index in vocab
4) craete the dataset avriable and give them the create class then after you create the dataoader and store the numrical data of question and answer in dataloader variable using the torch util package Dataloader

## RNN Architecture

In [59]:
import torch.nn as nn

class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embeding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True) # every time you implement the RNN it is mandatory to add batch_first=True
    self.out = nn.Linear(64, vocab_size)


  def forward(self, question):
    embeded = self.embeding(question)
    hidden, final = self.rnn(embeded) # we getting the tuple of len 2
    output = self.out(final.squeeze(0)) # before you get the tensor like [1, 1, 337] to [1, 337]

    return output

In [60]:
x = nn.Embedding(337, embedding_dim=50)

In [61]:
a = x(dataset[15][0])

In [62]:
y = nn.RNN(50, 64)

In [63]:
y(a)[0].shape

torch.Size([8, 64])

In [64]:
y(a)[1].shape

torch.Size([1, 64])

In [65]:
# Hidden Sates
y(a)[0]

tensor([[-0.1532,  0.1189,  0.0731,  0.0947, -0.2134, -0.5450,  0.0048, -0.7195,
         -0.5346,  0.0391,  0.0524,  0.1200,  0.2588,  0.6805, -0.2770, -0.0393,
          0.1188,  0.0440,  0.3582, -0.1315, -0.7036, -0.2045, -0.0102,  0.5497,
          0.1679, -0.1507,  0.6358,  0.7161, -0.2386, -0.3418, -0.6506,  0.5275,
          0.6777,  0.6981, -0.7300, -0.0194,  0.1244,  0.0634, -0.1682, -0.2559,
         -0.0388,  0.5277,  0.5769,  0.1251,  0.4823, -0.3796, -0.1933,  0.0304,
          0.3671,  0.0923, -0.4690,  0.0689,  0.6141, -0.7889,  0.1089,  0.7770,
         -0.1206,  0.1876, -0.2274,  0.4116,  0.4636, -0.1843,  0.0926, -0.4487],
        [-0.6733, -0.4834,  0.3602,  0.3614, -0.4027,  0.1780, -0.1883,  0.6186,
          0.7819, -0.1455,  0.0615,  0.5358,  0.1726, -0.4863, -0.1870,  0.2315,
         -0.0933, -0.3479, -0.4731,  0.6040,  0.6991,  0.2557,  0.2344,  0.4153,
          0.0071, -0.1869,  0.3840,  0.2884,  0.4733,  0.2834, -0.1471,  0.2865,
         -0.6623, -0.6707, 

In [66]:
# Final Output
y(a)[1]

# Here we get the two ouputs from RNN() so we cann't use the sequential layer here that's why you have to write the manually the forward function..

tensor([[-0.7410,  0.2258, -0.3763,  0.7941,  0.3092, -0.4885,  0.0438,  0.8101,
          0.4896,  0.2851, -0.3265, -0.6994,  0.0989, -0.0729, -0.6138,  0.2746,
          0.6511,  0.7484,  0.2499, -0.3189,  0.0856,  0.2479, -0.2135,  0.2840,
         -0.2909, -0.4387,  0.8752,  0.0458,  0.3222, -0.6270, -0.0900,  0.1259,
         -0.1841, -0.3759, -0.0324, -0.0518,  0.6834, -0.1501,  0.8314,  0.5132,
         -0.6298,  0.7410,  0.2347,  0.4476, -0.1169,  0.6789, -0.1795, -0.1569,
         -0.0393,  0.2474,  0.1510,  0.8312,  0.8122, -0.5584,  0.1120,  0.7374,
          0.2736,  0.0445,  0.6828, -0.0079,  0.2009, -0.4644,  0.4373, -0.2661]],
       grad_fn=<SqueezeBackward1>)

In [67]:
b = y(a)[1]

In [68]:
z = nn.Linear(64, 337)

In [69]:
z(b).shape

# For each word in your vacab you are getting the proabbility, And you select those word that are higher probbability

torch.Size([1, 337])

In [70]:
# Imporatant Variable
learning_rate = 0.001
epochs = 20

In [71]:
model = SimpleRNN(len(vocab))

In [72]:
# Create the Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [74]:
# Traning Loop
for epoch in range(epochs):

  total_loss = 0

  for question, ans in dataloader:

    optimizer.zero_grad()

    # Forward pass
    prediction = model(question)

    # Loss => when you going to calculate the loss it should be like your output shape (1, 324) - (1) for every questions answer
    # what was the problem is our shape is not comming correct
    loss = criterion(prediction, ans.squeeze(1)) # Flatten the target tensor

    # Gradient
    loss.backward()

    # update
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 527.657507
Epoch: 2, Loss: 461.169780
Epoch: 3, Loss: 381.245202
Epoch: 4, Loss: 317.316531
Epoch: 5, Loss: 265.750501
Epoch: 6, Loss: 218.847295
Epoch: 7, Loss: 175.432518
Epoch: 8, Loss: 138.029305
Epoch: 9, Loss: 106.208721
Epoch: 10, Loss: 81.125805
Epoch: 11, Loss: 62.466459
Epoch: 12, Loss: 48.491076
Epoch: 13, Loss: 38.684943
Epoch: 14, Loss: 31.315406
Epoch: 15, Loss: 25.784509
Epoch: 16, Loss: 21.584440
Epoch: 17, Loss: 18.271634
Epoch: 18, Loss: 15.605916
Epoch: 19, Loss: 13.457724
Epoch: 20, Loss: 11.570375


In [79]:
ans.squeeze(1)

tensor([186])

In [87]:
# Predictions

def predict(model, question, threshold = 0.5):

  # Question to Number
  num_ques = word_to_index(question, vocab)

  # Tensor
  Q_tens = torch.tensor(num_ques).unsqueeze(0) # get into perfect shape

  # Send to Model
  out = model(Q_tens)

  # print(out.shape) # => Gives you Logits - [1, 337]

  # convert logit to probabiliy
  probs = torch.nn.functional.softmax(out, dim=1)

  # Find max probs
  value, index_max = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index_max])

In [88]:
predict(model, "What is the smallest country in the world?")

vaticancity
